# Знакомимся с новыми данными: коронавирус

Мы будем изучать методы визуализации, используя два набора данных:

Обратите внимание, что в приведённом источнике данные о вакцинации постоянно обновляются. В данном модуле будет использоваться файл, который мы скачали заранее. Нажмите на "covid_data" и "country_vaccinations", чтобы скачать данные.

covid_data — данные о показателях заболеваемости за каждый день в различных регионах 187 стран (источник данных)

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

covid_data = pd.read_csv('data/covid_data.csv')
display(covid_data.head())

,date,province/state,country,confirmed,deaths,recovered
0,01/22/2020,Anhui,China,1.0,0.0,0.0
1,01/22/2020,Beijing,China,14.0,0.0,0.0
2,01/22/2020,Chongqing,China,6.0,0.0,0.0
3,01/22/2020,Fujian,China,1.0,0.0,0.0
4,01/22/2020,Gansu,China,0.0,0.0,0.0


Мы будем работать со следующими столбцами:

* date — дата наблюдения;
* province/state — наименование провинции/штата;
* country — наименование страны;
* confirmed — общее число зафиксированных случаев на указанный день;
* deaths — общее число зафиксированных смертей на указанный день;
* recovered — общее число выздоровлений на указанный день.

country_vaccinations — данные о процессе вакцинирования людей в 221 стране мира (источник данных)

In [4]:
vaccinations_data = pd.read_csv('data/country_vaccinations.csv')
vaccinations_data = vaccinations_data[
    ['country', 'date', 'total_vaccinations', 
        'people_vaccinated', 'people_vaccinated_per_hundred',
        'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
        'daily_vaccinations', 'vaccines']
]
vaccinations_data.head()

,country,date,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines
0,Afghanistan,2021-02-22,0.0,0.0,0.0,NaN,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
1,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
2,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
3,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
4,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."


Данная таблица содержит следующие столбцы:

* country — наименование страны;
* date — дата наблюдения;
* total_vaccinations — общее число введённых вакцин в стране на указанный день;
* people_vaccinated — общее число привитых первым компонентом в стране на указанный день;
* people_vaccinated_per_hundred — процент привитых первым компонентом в стране на указанный день (рассчитывается как people_vaccinated/population *100);
* people_fully_vaccinated — общее число привитых вторым компонентом в стране на указанный день (первый компонент уже был введён им ранее);
* people_fully_vaccinated_per_hundred — процент привитых вторым компонентом в стране на указанный день (рассчитывается как people_fully_vaccinated /population *100);
* daily_vaccination — ежедневная вакцинация (число вакцинированных в указанный день);
* vaccines — комбинации вакцин, используемые в стране.

### Немного об особенностях данных

В таблице covid_data данные о заболеваемости представлены для каждого региона (штата) страны за каждый день. Например, для США (United States) записи за 29 мая 2021 года имеют вид:

![](dst3-u1-md13_3_4.png)

В таблице vaccinations_data параметры вакцинирования отображены для всех стран на каждый день, без учёта регионов. Вот часть таблицы для тех же Соединённых Штатов за 29 мая 2021 года. Как видите, она всего одна:

![](dst3-u1-md13_3_5.png)

 данных много пропусков, а также неверно введённых значений. Например, для США (United States) и Великобритании (United Kingdom) в большинстве записей таблицы covid_data число выздоровевших людей равно 0. В данном модуле мы закроем на это глаза, но вообще эти страны подлежали бы исключению из данных, или нам понадобился бы дополнительный источник данных, из которого мы смогли бы получить недостающую информацию.

Обратите внимание, что признаки confirmed, deaths, recovered, total_vaccination, people_vaccinated, people_fully_vaccinated — это суммарные показатели по стране, то есть с каждым днём они должны расти. Такие признаки называют накопительными.

Возникает большое желание объединить таблицы. Для этого необходимо учитывать следующие нюансы:

В таблице covid_data необходимо предварительно рассчитать суммарное ежедневное число заболевших во всех провинциях/штатах в каждой стране.

В таблицах не совпадает число стран, а иногда и их названия. 

При объединении таблиц по столбцу мы определённо теряем данные (в данной задаче потери незначительны). Избежать этого можно ручными преобразованиями данных — искать различия в названиях стран в таблицах и преобразовывать их. Однако это не является темой данного модуля.

Таблицы имеют разные периоды наблюдений (вакцины появились позже, чем сам вирус). Объединив данные с типом inner, мы можем потерять большое количество наблюдений в таблице covid_data.

### Предобработка данных

Опираясь на замечания выше, выполним небольшую предобработку.

В таблице covid_data:

Группируем таблицу по дате и названию страны и рассчитываем суммарные показатели по всем регионам. Тем самым переходим от данных по регионам к данным по странам:

In [7]:
# Группируем таблицу по дате и названию страны и рассчитываем суммарные показатели по всем регионам. Тем самым переходим от данных по регионам к данным по странам:

covid_data = covid_data.groupby(
    ['date', 'country'], 
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()

In [8]:
# Преобразуем даты в формат datetime с помощью функции pd.to_datetime():

covid_data['date'] = pd.to_datetime(covid_data['date'])

In [9]:
# Создадим признак больных на данный момент (active). Для этого вычтем из общего числа зафиксированных случаев число смертей и число выздоровевших пациентов:

covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']

In [11]:
# Создадим признак ежедневного прироста числа заболевших, умерших и выздоровевших людей. Для этого отсортируем данные по названиям стран, а затем по датам. После этого произведём группировку по странам и рассчитаем разницу между «вчера и сегодня» с помощью метода diff():

covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()

In [12]:
# Итоговый вид таблицы covid_data:
display(covid_data)

,date,country,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered
881,2020-02-24,Afghanistan,1.0,0.0,0.0,1.0,NaN,NaN,NaN
920,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
963,2020-02-26,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1013,2020-02-27,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1067,2020-02-28,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2373,2020-03-12,occupied Palestinian territory,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2647,2020-03-14,occupied Palestinian territory,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2796,2020-03-15,occupied Palestinian territory,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2954,2020-03-16,occupied Palestinian territory,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# В таблице vaccinations_data достаточно будет преобразовать столбцы в формат datetime:

vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])

In [17]:
result_tab = pd.merge(covid_data, vaccinations_data, on='date', how='inner')
display(result_tab)

,date,country_x,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered,country_y,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines
0,2020-12-02,Afghanistan,46718.0,1841.0,37218.0,7659.0,202.0,19.0,272.0,Norway,0.0,0.0,0.00,NaN,NaN,NaN,"Moderna, Pfizer/BioNTech"
1,2020-12-03,Afghanistan,46837.0,1846.0,37260.0,7731.0,119.0,5.0,42.0,Norway,0.0,0.0,0.00,NaN,NaN,0.0,"Moderna, Pfizer/BioNTech"
2,2020-12-04,Afghanistan,46837.0,1846.0,37260.0,7731.0,0.0,0.0,0.0,Latvia,1.0,1.0,0.00,NaN,NaN,NaN,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ..."
3,2020-12-04,Afghanistan,46837.0,1846.0,37260.0,7731.0,0.0,0.0,0.0,Norway,0.0,0.0,0.00,NaN,NaN,0.0,"Moderna, Pfizer/BioNTech"
4,2020-12-05,Afghanistan,47072.0,1864.0,37393.0,7815.0,235.0,18.0,133.0,Latvia,NaN,NaN,NaN,NaN,NaN,0.0,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4285033,2021-05-29,Zimbabwe,38933.0,1594.0,36578.0,761.0,15.0,2.0,15.0,Wales,3253356.0,2141427.0,67.55,1111929.0,35.08,22975.0,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech"
4285034,2021-05-29,Zimbabwe,38933.0,1594.0,36578.0,761.0,15.0,2.0,15.0,Wallis and Futuna,NaN,NaN,NaN,NaN,NaN,13.0,Oxford/AstraZeneca
4285035,2021-05-29,Zimbabwe,38933.0,1594.0,36578.0,761.0,15.0,2.0,15.0,Yemen,104070.0,104070.0,0.34,NaN,NaN,4276.0,Oxford/AstraZeneca
4285036,2021-05-29,Zimbabwe,38933.0,1594.0,36578.0,761.0,15.0,2.0,15.0,Zambia,147033.0,141747.0,0.75,5286.0,0.03,1743.0,"Oxford/AstraZeneca, Sinopharm/Beijing"


## Задачи:

Задание 1

За какой период представлены данные в таблице covid_data? В качестве ответа введите даты в формате datetime (без указания времени).
Введите ответ в следующем формате (без пробелов): ГГГГ-ММ-ДД-ГГГГ-ММ-ДД

In [16]:
# Преобразуем столбец с датами в формат datetime (если это ещё не сделано)
covid_data['date'] = pd.to_datetime(covid_data['date'])

# Найдем минимальную и максимальную даты
start_date = covid_data['date'].min().strftime('%Y-%m-%d')
end_date = covid_data['date'].max().strftime('%Y-%m-%d')

# Выводим результат в нужном формате
result = f"{start_date}-{end_date}"
print(result)

2020-01-22-2021-05-29


Задание 2

За какой период представлены данные в таблице vaccinations_data? В качестве ответа введите даты в формате datetime без указания времени.
Введите ответ в следующем формате (без пробелов): ГГГГ-ММ-ДД-ГГГГ-ММ-ДД

In [18]:
# Преобразуем столбец с датами в формат datetime (если это ещё не сделано)
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])

# Найдем минимальную и максимальную даты
start_date = vaccinations_data['date'].min().strftime('%Y-%m-%d')
end_date = vaccinations_data['date'].max().strftime('%Y-%m-%d')

# Выводим результат в нужном формате
result = f"{start_date}-{end_date}"
print(result)

2020-12-02-2021-09-06


Задание 3

С помощью метода merge() объедините таблицы covid_data и vaccinations_data по столбцам date и country.

Тип объединения выставьте так, чтобы в результирующую таблицу попали только наблюдения за период, вычисленный в задании 3.1. То есть в результирующую таблицу должны попасть все записи из таблицы covid_data и из её пересечения с vaccinations_data, но не более. Результат объединения занесите в переменную covid_df.

Сохраните таблицу covid_df — она понадобится нам в следующем юните.

Сколько строк и столбцов в таблице covid_df?
Введите ответ в виде двух чисел через дефис (например, 333-33): первое число — количество строк, второе число — количество столбцов.

In [20]:
# Преобразуем столбцы с датами в формат datetime (если это ещё не сделано)
covid_data['date'] = pd.to_datetime(covid_data['date'])
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])

# Определяем минимальную и максимальную даты для фильтрации
start_date = covid_data['date'].min()
end_date = covid_data['date'].max()

# Фильтруем таблицы по этим датам
covid_data_filtered = covid_data[(covid_data['date'] >= start_date) & (covid_data['date'] <= end_date)]
vaccinations_data_filtered = vaccinations_data[(vaccinations_data['date'] >= start_date) & (vaccinations_data['date'] <= end_date)]

# Объединяем таблицы по столбцам 'date' и 'country'
covid_df = pd.merge(covid_data_filtered, vaccinations_data_filtered, on=['date', 'country'], how='left')

# Выводим результат
print(covid_df.head())

        date      country  confirmed  deaths  recovered  active  \
0 2020-02-24  Afghanistan        1.0     0.0        0.0     1.0   
1 2020-02-25  Afghanistan        1.0     0.0        0.0     1.0   
2 2020-02-26  Afghanistan        1.0     0.0        0.0     1.0   
3 2020-02-27  Afghanistan        1.0     0.0        0.0     1.0   
4 2020-02-28  Afghanistan        1.0     0.0        0.0     1.0   

   daily_confirmed  daily_deaths  daily_recovered  total_vaccinations  \
0              NaN           NaN              NaN                 NaN   
1              0.0           0.0              0.0                 NaN   
2              0.0           0.0              0.0                 NaN   
3              0.0           0.0              0.0                 NaN   
4              0.0           0.0              0.0                 NaN   

   people_vaccinated  people_vaccinated_per_hundred  people_fully_vaccinated  \
0                NaN                            NaN                      NaN  

In [22]:
# Сохраняем результат в файл CSV
covid_df.to_csv('covid_df.csv', index=False)

In [23]:
# Получаем количество строк и столбцов
rows, columns = covid_df.shape
print(f"{rows}-{columns}")

86785-16


В получившейся в задании 3 таблице covid_df создайте признаки death_rate — общий процент смертей среди зафиксированных случаев (летальность) и recover_rate — общий процент случаев выздоровления. Данные характеристики рассчитайте как отношение числа смертей (deaths) и числа выздоровлений (recovered) к числу зафиксированных случаев (confirmed) и умножьте результаты на 100%.

In [24]:
# Избегаем деления на ноль, заменяя 0 в столбце 'confirmed' на NaN
covid_df['death_rate'] = (covid_df['deaths'] / covid_df['confirmed'].replace(0, float('nan'))) * 100
covid_df['recover_rate'] = (covid_df['recovered'] / covid_df['confirmed'].replace(0, float('nan'))) * 100

In [25]:
# Рассчитываем death_rate и recover_rate
covid_df['death_rate'] = (covid_df['deaths'] / covid_df['confirmed']) * 100
covid_df['recover_rate'] = (covid_df['recovered'] / covid_df['confirmed']) * 100

# Выводим результат
print(covid_df[['death_rate', 'recover_rate']].head())

   death_rate  recover_rate
0         0.0           0.0
1         0.0           0.0
2         0.0           0.0
3         0.0           0.0
4         0.0           0.0


Задание 4

Какова максимальная летальность в США (United States) за весь период? Ответ округлите до второго знака после запятой.

In [26]:
# Фильтруем данные по США
us_data = covid_df[covid_df['country'] == 'United States']

# Находим максимальную летальность и округляем до 2 знаков
max_death_rate_us = us_data['death_rate'].max()
max_death_rate_us_rounded = round(max_death_rate_us, 2)

# Выводим результат
print(max_death_rate_us_rounded)

7.19


Задание 5

Чему равен средний процент выздоровевших в России (Russia)? Ответ округлите до второго знака после запятой.

In [27]:
# Фильтруем данные по России
russia_data = covid_df[covid_df['country'] == 'Russia']

# Находим средний процент выздоровевших и округляем до 2 знаков
avg_recover_rate_russia = russia_data['recover_rate'].mean()
avg_recover_rate_russia_rounded = round(avg_recover_rate_russia, 2)

# Выводим результат
print(avg_recover_rate_russia_rounded)

67.06
